In [1]:
import torch
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import geopandas as gpd
import datetime
import matplotlib
from itertools import product
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import scipy

import sys
sys.path.insert(0, '/Users/sunghoonyang/PycharmProjects/noise-capstone/analysis/311/duke')

from datetime import datetime
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [7]:
df = pd.read_csv('/Users/sunghoonyang/Google Drive/Noise Capstone/data/AfterHourVariance_lat_long.csv')
df.head()

/Users/sunghoonyang/PycharmProjects/ds_workspace/Noise Capstone/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,timestamp,bbl,borough,block,lot,bin,premises,jobno,referenceno,permitno,jobtype,efiled,feeexempt,filingtype,daysbilled,status,totalfee,houseno,streetname,floors,cbno,contractorname,contractorbusinessname,contractorbusinessaddr,contractorlicensetype,contractorphone,contractorlicense,residencewithin200feet,workwithinenclosedbuilding,fullorpatialdemolition,craneuse,daysrequested,daysapproved,approvedfor,erenew,applyreason,approved,erenewalauthorized,description,range,count,centroid,lon,lat
0,2011-02-06 10:00:00-05:00,1012880056,MANHATTAN,1288,56,1088582,14 EAST 53 STREET MANHATTAN,104747115,100,104747115,NB,NaN,No,Initial,2,AHV SUCCESSFULLY ISSUED,$260.00,510,MADISON AVENUE,1,105.0,WILLIAM MOTHERWAY,TISHMAN CONSTRUCTION CORP,666 FIFTH AVENUE NEW YORK NY 10103,GC,631-724-5915,2540,False,False,False,False,2,2,"02/05/2011,Saturday,10:00 AM,5:00 PM;02/06/201...",Yes,PUBLIC SAFETY,DEBRIS REMOVAL,"Saturday 10:00 AM to 5:00 PM, Sunday 10:00 AM ...",NaN,"[{u'start': 2011-02-05 10:00:00-05:00, u'end':...",1,"(40.7596705219, -73.9747348344)",40.759671,-73.974735
1,2011-02-10 18:00:00-05:00,1007870044,MANHATTAN,787,44,1014460,508 7 AVENUE MANHATTAN,120537348,400009,120537348,A2,NaN,No,Initial,3,AHV SUCCESSFULLY ISSUED,$340.00,512,SEVENTH AVENUE,"33, 34",105.0,BRIAN GREEN,ARTISAN CONSTR PARTNERS L,"65 WEST 37TH STREET, #300 NEW YORK NY 10018",GC,212-840-8500,603945,False,False,False,False,3,3,"02/08/2011,Tuesday,6:00 PM,12:00 AM;02/09/2011...",Yes,PUBLIC SAFETY,BUSINESS HOURS OPERATIONS INTERRUPTION,"Tuesday 6:00 PM to 12:00 AM, Wednesday 6:00 PM...",REMOVE INTERIOR PARTITIONS,"[{u'start': 2011-02-08 18:00:00-05:00, u'end':...",1,"(40.7534042442, -73.9893360555)",40.753404,-73.989336
2,2011-02-10 18:00:00-05:00,1005540001,MANHATTAN,554,1,1008952,770 BROADWAY MANHATTAN,120419608,400173,120419608,A2,NaN,No,Initial,3,AHV SUCCESSFULLY ISSUED,$340.00,770,BROADWAY,"10, 11, 12",102.0,DAVID STORY,TRINITY BUILDING AND CONS,1 JEWEL DRIVE #322 WILMINGTON MA 01887,GC,781-938-0008,37220,False,False,False,False,3,3,"02/09/2011,Wednesday,6:00 PM,3:00 AM;02/10/201...",Yes,PUBLIC SAFETY,BUSINESS HOURS OPERATIONS INTERRUPTION,"Wednesday 6:00 PM to 3:00 AM, Thursday 6:00 PM...","STEEL WELDING, SITE CLEAN UP","[{u'start': 2011-02-09 18:00:00-05:00, u'end':...",1,"(40.7306014266, -73.9913758691)",40.730601,-73.991376
3,2011-02-10 18:00:00-05:00,1013090001,MANHATTAN,1309,1,1036476,405 PARK AVENUE MANHATTAN,120550849,400064,120550849,A2,NaN,No,Initial,2,AHV SUCCESSFULLY ISSUED,$260.00,405,PARK AVENUE,12,105.0,BRIAN ROTHSCHILD,ENVIROCHROME INTERIORS IN,80 PARK AVENUE NEW YORK NY 10016,GC,212-687-7330,19046,False,False,False,False,2,2,"02/10/2011,Thursday,6:00 PM,6:00 AM;",Yes,PUBLIC SAFETY,BUSINESS HOURS OPERATIONS INTERRUPTION,Thursday 6:00 PM to 6:00 AM,"MINOR DEMOLITION OF PARTITIONS, DOORS, SUSPEND...","[{u'start': 2011-02-10 18:00:00-05:00, u'end':...",1,"(40.7597358049, -73.9716172608)",40.759736,-73.971617
4,2011-02-11 18:00:00-05:00,1017190001,MANHATTAN,1719,1,1083967,18 MT MORRIS PARK WEST MANHATTAN,120088939,400013,120088939,A2,NaN,No,Initial,4,AHV SUCCESSFULLY ISSUED,$520.00,18,MT. MORRIS PARK WEST,1,111.0,JOHN DRISCOU,TRITON STRUCTURAL CONCRET,15435 INNOVATION DR #225 SAN DIEGO CA 92121,GC,347-423-6618,604939,False,False,False,False,4,4,"02/08/2011,Tuesday,6:00 PM,12:00 AM;02/09/2011...",Yes,PUBLIC SAFETY,TRAFFIC CONGESTION (PEDESTRIAN AND/OR VEHICULAR),"Tuesday 6:00 PM to 12:00 AM, Wednesday 6:00 PM...","MOVING OF MATERIALS, INSTALLATION OF ROOFING S...","[{u'start': 2011-02-08 18:00:00-05:00, u'end':...",1,"(40.8045998935, -73.943228267)",40.804600,-73.943228


In [8]:
__DATA_FILE_PATH__ = r'/Users/sunghoonyang/Google Drive/Noise Capstone/data/dep_ntanm_noise_only_311_data/NTANM_DEP_NZONLY_311_Service_Requests_from_2010_to_Present.csv'
def _convert_dt(dt):
    try:
        return pd.to_datetime(dt, format='%Y-%m-%d %H:%M:%S')
    except:
        return np.nan


__RAW_DF__ = pd.read_csv(
    __DATA_FILE_PATH__
    , converters={k: _convert_dt for k in ['Created Date']}
)
__RAW_DF__.head()

/Users/sunghoonyang/PycharmProjects/ds_workspace/Noise Capstone/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Address Type,City,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Latitude,Longitude,Location,geom,boro_code,boro_name,county_fip,ntacode,ntaname,shape_area,shape_leng
0,2017-09-12 18:21:00,2017-09-15 22:30:00,DEP,Department of Environmental Protection,Noise,Noise: air condition/ventilation equipment (NV1),11216.0,1232 BEDFORD AVENUE,BEDFORD AVENUE,HANCOCK ST,HALSEY ST,ADDRESS,BROOKLYN,Closed,NaN,The Department of Environmental Protection det...,2017-09-15 22:30:00,03 BROOKLYN,3.020000e+09,BROOKLYN,997116.0,187685.0,ONLINE,Unspecified,BROOKLYN,40.681819,-73.953612,"(40.68181938473442, -73.95361203646364)",POINT (-73.95361203646364 40.68181938473442),3.0,Brooklyn,47,BK69,Clinton Hill,2.052820e+07,23971.465822
1,2017-09-13 14:58:00,2017-09-13 15:36:00,DEP,Department of Environmental Protection,Noise,"Noise, Barking Dog (NR5)",11205.0,219 WASHINGTON AVENUE,WASHINGTON AVENUE,MYRTLE AVE,WILLOUGHBY AVE,ADDRESS,BROOKLYN,Closed,NaN,The Department of Environmental Protection det...,2017-09-13 15:36:00,02 BROOKLYN,3.019040e+09,BROOKLYN,993497.0,191488.0,ONLINE,Unspecified,BROOKLYN,40.692262,-73.966655,"(40.6922622678706, -73.96665500760106)",POINT (-73.96665500760106 40.6922622678706),3.0,Brooklyn,47,BK69,Clinton Hill,2.052820e+07,23971.465822
2,2017-09-13 03:13:00,2017-09-13 21:26:00,DEP,Department of Environmental Protection,Noise,Noise: Alarms (NR3),11238.0,101 QUINCY STREET,QUINCY STREET,CLASSON AVE,FRANKLIN AVE,ADDRESS,BROOKLYN,Closed,NaN,The Department of Environmental Protection res...,2017-09-13 21:26:00,03 BROOKLYN,3.019700e+09,BROOKLYN,996060.0,189277.0,PHONE,Unspecified,BROOKLYN,40.686191,-73.957417,"(40.68619053293611, -73.95741662072693)",POINT (-73.95741662072693 40.68619053293611),3.0,Brooklyn,47,BK69,Clinton Hill,2.052820e+07,23971.465822
3,2017-09-14 18:55:00,2017-09-19 10:34:00,DEP,Department of Environmental Protection,Noise,"Noise, Barking Dog (NR5)",11238.0,510 WASHINGTON AVENUE,WASHINGTON AVENUE,GATES AVE,FULTON ST,ADDRESS,BROOKLYN,Closed,NaN,The Department of Environmental Protection det...,2017-09-19 10:34:00,02 BROOKLYN,3.019780e+09,BROOKLYN,993977.0,188377.0,ONLINE,Unspecified,BROOKLYN,40.683723,-73.964929,"(40.68372277301225, -73.96492860317981)",POINT (-73.96492860317981 40.68372277301225),3.0,Brooklyn,47,BK69,Clinton Hill,2.052820e+07,23971.465822
4,2017-09-15 06:06:00,2017-09-29 07:00:00,DEP,Department of Environmental Protection,Noise,Noise: Construction Before/After Hours (NM1),11238.0,1068 FULTON STREET,FULTON STREET,CLASSON AVE,CLAVER PL,ADDRESS,BROOKLYN,Closed,NaN,The Department of Environmental Protection did...,2017-09-29 07:00:00,03 BROOKLYN,3.020160e+09,BROOKLYN,995861.0,187619.0,ONLINE,Unspecified,BROOKLYN,40.681640,-73.958137,"(40.68163996496247, -73.95813701299446)",POINT (-73.95813701299446 40.68163996496247),3.0,Brooklyn,47,BK69,Clinton Hill,2.052820e+07,23971.465822
